In [ ]:
import numpy as np 
import pandas as pd 
import pymysql
from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from pandas import ExcelWriter
from pandas import ExcelFile
from sklearn import preprocessing
%matplotlib notebook
import json
import plotly
import matplotlib.pyplot as plt, mpld3
from matplotlib import rcParams
from matplotlib.cm import rainbow
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import State, Input, Output
import dash_bootstrap_components as dbc
import dash_auth
import plotly.express as px
from flask import Flask, redirect
from pathlib import Path
import dash_table
import plotly.graph_objs as go
import plotly.figure_factory as ff
import seaborn as sn
engine = create_engine('mysql+pymysql://root:@localhost/student')
con = engine.connect()
colors = ['crimson','lightslategray','yellow','blue','red','pink','green'] 
df = pd.read_sql_query("select * from results", con)
le = preprocessing.LabelEncoder()
df2 = df.apply(le.fit_transform)
#fig = df2.boxplot(column=['subject','mid1','mid2'], by='GRADE',layout=(3, 1))
 
Grade = go.Box(
    y=df2['GRADE'],
    name='Grade'
)
Mid1 = go.Box(
    y=df2['mid1'],
    name='Mid1 Marks'
)
Mid2= go.Box(
    y=df2['mid2'],
    name='Mid2 Marks'
)

data = [Grade, Mid1,Mid2]
Mid1=df['mid2']
Mid2=df['mid1']
#fig = go.Figure(data=go.Scatter(x=t, y=y, mode='markers'))
data2=[go.Scatter(x=Mid1,y=Mid2,mode='markers+text')]
layout1 = go.Layout(
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='Mid1',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='Mid2',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    )
)
fig0 = go.Figure(data=data2,layout=layout1)
del df2['index']
cor=df2.corr()
#print(cor)
data3=[go.Heatmap(z=[cor['subject'],cor['mid1'],cor['mid2'],cor['GRADE']],autocolorscale=True)]
figa=[go.Barpolar(theta=df2['subject'], r=df2['GRADE'])]
trace3 = go.Scatter(
    x=df2['GRADE'], y=df2['subject'], mode='markers', name='points',
    marker=dict(color='rgb(102,0,0)', size=2, opacity=0.4)
)
trace4 = go.Histogram2dContour(
    x=df2['GRADE'], y=df2['subject'], name='density', ncontours=20,
    colorscale='Hot', reversescale=True, showscale=False
)
trace5 = go.Histogram(
    x=df2['GRADE'], name='x density',
    marker=dict(color='rgb(102,0,0)'),
    yaxis='y2'
)
trace6 = go.Histogram(
    y=df2['subject'], name='y density', marker=dict(color='rgb(102,0,0)'),
    xaxis='x2'
)
layout = go.Layout(
    showlegend=False,
    autosize=False,
    width=700,
    height=550,
    xaxis=dict(
        domain=[0, 0.85],
        showgrid=False,
        zeroline=False
    ),
    yaxis=dict(
        domain=[0, 0.85],
        showgrid=False,
        zeroline=False
    ),
    margin=dict(
        t=50
    ),
    hovermode='closest',
    bargap=0,
    xaxis2=dict(
       
        domain=[0.85, 1],
        showgrid=False,
        zeroline=False
    ),
    yaxis2=dict(
        
        domain=[0.85, 1],
        showgrid=False,
        zeroline=False
    )
)
data5 = [trace3, trace4, trace5, trace6]
fig5 = go.Figure(data=data5,layout=layout)
balance_data = pd.read_excel('dataset.xlsx') 
X = balance_data.values[:, 2:4] 
Y = balance_data.values[:, 4]
X_train, X_test, y_train, y_test = train_test_split(  X, Y, test_size = 0.3, random_state = 100)
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100, min_samples_leaf=2) 
clf_gini.fit(X_train, y_train) 
y_pred = clf_gini.predict(X_test) 
acc=accuracy_score(y_test,y_pred)*100
cla=classification_report(y_test, y_pred,labels=np.unique(y_pred))
conf=pd.DataFrame(confusion_matrix(y_test, y_pred))
#data4=[go.Heatmap(conf,annot=True,cmap='RdYlGn')]
data4=[go.Heatmap(z=[conf[0],conf[1],conf[2],conf[3],conf[4],conf[5],conf[6]],autocolorscale=True)]
datafra = [['0','1.00','1.00','1.00','6'], ['1', '1.00','1.00','1.00','29'], ['2','1.00','1.00','1.00','56'],['3','1.00','1.00','1.00','40'],['4','1.00','1.00','1.00','9'],['5','1.00','1.00','1.00','5'],['6','1.00','1.00','1.00','5'],['micro avg' ,'0.97' ,'1.00','0.98' ,'145'],['macro avg','0.92',' 1.00' ,'0.94','145'],['weighted avg','0.98','1.00','0.99','145']]
repo = pd.DataFrame(datafra, columns = ['  ' ,  'precision'  ,  'recall','f1-score' ,  'support'])

app21 = dash.Dash(external_stylesheets=[dbc.themes.CERULEAN])
app21.config['suppress_callback_exceptions']=True
app21.layout = html.Div(style={'backgroundColor':'white'},children=[
        html.H1('STUDENT PERFORMANCE ANALYSIS SYSTEM USING MACHINE LEARNING',style={'text':'bold','textAlign': 'center','color':'red'}),
        html.Br(),html.Br(),
        html.Div([html.A('HOME PAGE', href='http://127.0.0.1:2020/', target='_self')]),html.Br(),
        html.Div(style={'textAlign': 'center','text':'bold'},children=[
        dbc.Label("Accuracy is  96.66666666666667"),html.Br(),html.Br(),
        dbc.Label("Report : "),html.Br(),html.Br(),
        dbc.Table.from_dataframe(repo, striped=True, bordered=True, hover=True),
        html.Br(),html.Br(),
        dbc.Label("Select Chart "),
        dcc.Dropdown(
        id='d3' ,style={'vertical-align': 'middle','display': 'inline-block','height': '30px', 'width': '200px'},
        options=[
            {'label': 'Boxplot', 'value': 'box'},
            {'label': 'Scatterplot', 'value': 'scat'},
            {'label': 'Correlation', 'value': 'cor'},
            {'label': 'Confusion', 'value': 'con'},
            {'label' : 'Bipolar', 'value': 'bi'},
            {'label' : '2Ddensityplot','value': '2d'}
        ],value='con'),html.Br(),
        html.Div(id='page1',style={'textAlign': 'center','backgroundColor':'white'})])
        ])
            
@app21.callback(
    dash.dependencies.Output('page1', 'children'),
    [dash.dependencies.Input('d3','value')],
    state=[State('d3','value')])
def update_options(d3,d3value):
        if(d3value=="box"):
            return html.Div([html.H3('Boxplot'),dcc.Graph(figure={'data': data}, id='box-plot')])
        if(d3value=="scat"):
            return html.Div([html.H3('Scatterplot'),dcc.Graph(figure=fig0)])
        if(d3value=="cor"):
            return html.Div([html.H3('Correlation'),dcc.Graph(figure={"data":data3})])
        if(d3value=="con"):
            return html.Div([html.H3('Confusion'),dcc.Graph(figure={"data":data4})])
        if(d3value=="bi"):
            return html.Div([html.H3('Bipolar'),dcc.Graph(figure={"data":figa})])
        if(d3value=="2d"):
            return html.Div([html.H3('2Ddensityplot'),dcc.Graph(figure=fig5)])
       
       
if __name__ == '__main__':
    app21.run_server(port=2014)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:2014/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Apr/2020 00:56:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:56:17] "GET /assets/stylesheet.css?m=1584007644.0875306 HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:56:17] "GET /_dash-component-suites/dash_renderer/react@16.v1_2_2m1583924370.8.6.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:56:17] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_2_2m1583924370.7.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:56:17] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_2_2m1583924370.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:56:17] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_2_2m1583924370.8.6.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:56:17] "GET /_dash-component-suites/dash_table/bundle.v4_6_1m1583924377.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:56:17] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1